### PRACTICA 5 FILTROS 😁 

1 Filtro Modo Hamburguesas

In [1]:
import cv2
import mediapipe as mp
from mediapipe.tasks.python import vision
from mediapipe.tasks.python import BaseOptions
import numpy as np
import math
import random
import time


# Cargar imagen del emoji de hamburguesa y del corazon
burger_img = cv2.imread('images/burger_emoji.png', cv2.IMREAD_UNCHANGED)
heart_img = cv2.imread('images/heart_emoji.png', cv2.IMREAD_UNCHANGED)
hat_img = cv2.imread('images/hat_img.png', cv2.IMREAD_UNCHANGED)

burger_positions = []

def draw_burgers(distance, lip_top, frame):
    frame_height, frame_width = frame.shape[0], frame.shape[1]
    
    if distance > 60:  # Umbral
        burger_resized = cv2.resize(burger_img, (50, 50))
        burger_width, burger_height = burger_resized.shape[1], burger_resized.shape[0]

        # Generar nuevas hamburguesas en la parte derecha e izquierda de la cara
        for i in range(5):  # Número de hamburguesas a mostrar
            if i >= len(burger_positions):
                center_x, center_y = frame_width // 2, frame_height // 2  # Coordenadas centrales del frame
                # Inicializar la posición de las hamburguesas al estilo json
                burger_positions.append({
                    "x": center_x + (i - 2) * (burger_width + 20),
                    "y": center_y - burger_height // 2,
                    "direction": 1,                    # Dirección hacia abajo
                    "side": "right"                    # Indicar que es del lado derecho
                })
                burger_positions.append({
                    "x": lip_top[0] - 100 - (i * 80),  # Espaciado entre hamburguesas en el lado izquierdo
                    "y": -burger_height,               # Comenzar desde la parte superior fuera de la vista
                    "direction": 1,                    # Dirección hacia abajo
                    "side": "left"                     # Indicar que es del lado izquierdo
                })
            
            # Actualizamos la posición de las hamburguesas
            for j in range(len(burger_positions)):
                burger_positions[j]["y"] += burger_positions[j]["direction"] * 5  # Mover hacia abajo
                if burger_positions[j]["y"] > frame_height: burger_positions[j]["y"] = -burger_height  # Volver a la parte superior

                # Verificar que la hamburguesa no se salga del cuadro
                x_offset, y_offset  = burger_positions[j]["x"], burger_positions[j]["y"]

                if x_offset + burger_width <= frame_width and y_offset + burger_height <= frame_height:
                    # Dibujar la hamburguesa en su nueva posición
                    for c in range(0, 3):
                        try:
                            frame[y_offset:y_offset+burger_height, x_offset:x_offset+burger_width, c] = \
                                burger_resized[:, :, c] * (burger_resized[:, :, 3] / 255.0) + \
                                frame[y_offset:y_offset+burger_height, x_offset:x_offset+burger_width, c] * \
                                (1.0 - burger_resized[:, :, 3] / 255.0)
                        except Exception as e:
                            continue
                            
def apply_hat(frame, face_landmarks):
    # Utilizamos los puntos 183 y 332 de la malla facial como referencia para la parte superior de la cabeza
    head_top = (int(face_landmarks.landmark[183].x * frame.shape[1]), 
                int(face_landmarks.landmark[332].y * frame.shape[0]))

    # Combinación de puntos (punto 183 y punto 332).
    head_top_avg_y = int((face_landmarks.landmark[183].y + face_landmarks.landmark[332].y) * frame.shape[0] / 2)

    x_offset = head_top[0] - hat_img.shape[1] // 2       # Centrado horizontalmente
    y_offset = head_top_avg_y - (hat_img.shape[0] // 2)  # Ajuste hacia la parte superior de la cabeza
    
    # Constante de desplazamiento
    y_offset -= 150

    # Asegurarnos de que la gorra no quede fuera de la imagen
    frame_height, frame_width = frame.shape[:2]
    
    # Evitar que x_offset se salga por la izquierda y que y_offset se salga por la parte superior
    if x_offset < 0: x_offset = 0
    if y_offset < 0: y_offset = 0

    # Asegurarnos de que la gorra no se salga por la derecha ni por la parte inferior
    if x_offset + hat_img.shape[1] > frame_width: x_offset = frame_width - hat_img.shape[1]

    if y_offset + hat_img.shape[0] > frame_height: y_offset = frame_height - hat_img.shape[0]

    # Verificar que la gorra está dentro de los límites del frame
    if x_offset + hat_img.shape[1] <= frame_width and y_offset + hat_img.shape[0] <= frame_height:
        # Obtener la región de interés (ROI) en el frame
        roi = frame[y_offset:y_offset + hat_img.shape[0], x_offset:x_offset + hat_img.shape[1]]

        # Asegurar que la gorra tiene un canal alfa para manejar la transparencia
        for c in range(0, 3):
            roi[:, :, c] = roi[:, :, c] * (1.0 - hat_img[:, :, 3] / 255.0) + \
                            hat_img[:, :, c] * (hat_img[:, :, 3] / 255.0)

        # Colocar la gorra en el frame
        frame[y_offset:y_offset + hat_img.shape[0], x_offset:x_offset + hat_img.shape[1]] = roi


In [2]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

def calculate_distance(p1, p2):
    # Calcular distancia euclidiana entre dos puntos (p1 y p2 son tuplas (x, y))
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)
           
# Función para obtener las coordenadas de los puntos clave de la malla facial
def process_frame(frame, face_mesh, points, mode):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    keypoints = []  # Lista para guardar las coordenadas de los puntos clave

    if results.multi_face_landmarks is not None:
        for face_landmarks in results.multi_face_landmarks:       
            # Dibujar y guardar los puntos específicos
            for point in points:
                if point < len(face_landmarks.landmark):  # Verificar índice válido
                    x = int(face_landmarks.landmark[point].x * frame.shape[1])
                    y = int(face_landmarks.landmark[point].y * frame.shape[0])
                    keypoints.append((x, y))  # Agregar las coordenadas a la lista
            if (mode == 2):
                apply_hat(frame, face_landmarks)
                
                # Calcular la distancia entre el labio superior e inferior (usando los puntos 13, 14, 18, 19)
                lip_top = (int(face_landmarks.landmark[13].x * frame.shape[1]), int(face_landmarks.landmark[14].y * frame.shape[0]))
                lip_bottom = (int(face_landmarks.landmark[18].x * frame.shape[1]), int(face_landmarks.landmark[19].y * frame.shape[0]))
                draw_burgers(calculate_distance(lip_top, lip_bottom), lip_top, frame)
    
    return keypoints


2 Filtro Modo Duende

In [3]:
# Inicializamos las mallas de media pipe
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Cargar las imágenes de las orejas
left_ear_img = cv2.imread('images/orejaI.png', cv2.IMREAD_UNCHANGED) 
right_ear_img = cv2.imread('images/orejaD.png', cv2.IMREAD_UNCHANGED)
emoji_img = cv2.imread('images/dinero.png', cv2.IMREAD_UNCHANGED) 

# Lista para almacenar los emojis de dinero
falling_emoji = []

# Clase para el dinero que cae
class FallingEmoji:
    def __init__(self, x, y, speed, time_to_live):
        self.x = x
        self.y = y
        self.speed = speed
        self.time_to_live = time_to_live  # Tiempo de vida del emoji
        self.creation_time = time.time()  # Momento en que se creó el emoji

    def update(self):
        self.y += self.speed  # Movimiento hacia abajo
        if self.y > 480:  # Si el emoji se sale de la pantalla, lo reubicamos en la parte superior
            self.y = 0
            self.x = random.randint(0, 640)

        # Verificar si el emoji ha excedido su tiempo de vida
        if time.time() - self.creation_time > self.time_to_live:
            return False  
        return True 
    
def euclidean_distance(p1, p2):
    return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)

def process_frame_mode1(frame, face_mesh):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    points_data = {}

    if results.multi_face_landmarks is not None:
        for face_landmarks in results.multi_face_landmarks:
            points = [234, 454, 0, 17]  # Añadimos los puntos para las orejas (234 y 454) y los puntos de la boca (0 y 17)

            for point in points:
                if point < len(face_landmarks.landmark):
                    x = face_landmarks.landmark[point].x
                    y = face_landmarks.landmark[point].y
                    z = face_landmarks.landmark[point].z

                    x_pixel = int(x * frame.shape[1])
                    y_pixel = int(y * frame.shape[0])

                    points_data[point] = {'x': x_pixel, 'y': y_pixel, 'z': z}

            # Colocamos las orejas (puntos 234 y 454 para la oreja izquierda y derecha)
            if 234 in points_data:
                left_ear_position = points_data[234]
                left_ear_resized = cv2.resize(left_ear_img, (100, 100))  # Aumentamos el tamaño de la oreja
                lx, ly = left_ear_position['x'], left_ear_position['y']

                # Ajustamos vertical y horizontalmente
                lx -= 80  
                ly -= 60 

                # Aseguramos de que la oreja no se salga de los limites
                if lx + left_ear_resized.shape[1] > frame.shape[1]:
                    lx = frame.shape[1] - left_ear_resized.shape[1]
                if ly + left_ear_resized.shape[0] > frame.shape[0]:
                    ly = frame.shape[0] - left_ear_resized.shape[0]

                # Superponemos la oreja izquierda en el frame
                for c in range(0, 3):  # Para las 3 capas (RGB)
                    if lx + left_ear_resized.shape[1] <= frame.shape[1] and ly + left_ear_resized.shape[0] <= frame.shape[0]:
                        frame[ly:ly + left_ear_resized.shape[0], lx:lx + left_ear_resized.shape[1], c] = \
                            left_ear_resized[:, :, c] * (left_ear_resized[:, :, 3] / 255.0) + \
                            frame[ly:ly + left_ear_resized.shape[0], lx:lx + left_ear_resized.shape[1], c] * (1.0 - left_ear_resized[:, :, 3] / 255.0)

            if 454 in points_data:
                right_ear_position = points_data[454]
                right_ear_resized = cv2.resize(right_ear_img, (100, 100))  # Aumentamos el tamaño de la oreja
                rx, ry = right_ear_position['x'], right_ear_position['y']

                # ajustamos la oreja al rosto
                rx -= 15  
                ry -= 60 

                # Aseguramos de que la oreja no se salga
                if rx + right_ear_resized.shape[1] > frame.shape[1]:
                    rx = frame.shape[1] - right_ear_resized.shape[1]
                if ry + right_ear_resized.shape[0] > frame.shape[0]:
                    ry = frame.shape[0] - right_ear_resized.shape[0]

                # Superponer la oreja derecha en el frame
                for c in range(0, 3):  # Para las 3 capas (RGB)
                    if rx + right_ear_resized.shape[1] <= frame.shape[1] and ry + right_ear_resized.shape[0] <= frame.shape[0]:
                        frame[ry:ry + right_ear_resized.shape[0], rx:rx + right_ear_resized.shape[1], c] = \
                            right_ear_resized[:, :, c] * (right_ear_resized[:, :, 3] / 255.0) + \
                            frame[ry:ry + right_ear_resized.shape[0], rx:rx + right_ear_resized.shape[1], c] * (1.0 - right_ear_resized[:, :, 3] / 255.0)

            # Calcular la distancia entre los puntos 0 y 17 para saber si la boca está abierta
            if 0 in points_data and 17 in points_data:
                mouth_open_distance = euclidean_distance(
                    (points_data[0]['x'], points_data[0]['y']),
                    (points_data[17]['x'], points_data[17]['y'])
                )

                # Umbral para la distancia 
                threshold = 40 

                probabilidad_generar_emoji = 0.1

                if mouth_open_distance > threshold and random.random() < probabilidad_generar_emoji:
                    if random.random() < 0.4:
                        new_emoji = FallingEmoji(random.randint(0, frame.shape[1] - 80), 0, random.randint(2, 5), time_to_live=5)
                        falling_emoji.append(new_emoji)
                                
                # Eliminamos segun va pasando el tiempo de vida
                falling_emoji[:] = [emoji for emoji in falling_emoji if emoji.update()]
            

            # Actualizamos la posición de los emojis que caen
            for emoji in falling_emoji:
                emoji.update()
                # Redimensionamos la imagen del emoji
                emoji_resized = cv2.resize(emoji_img, (80, 50))

                # Superponemos el emoji sobre el frame
                for c in range(0, 3):
                    if emoji.y + emoji_resized.shape[0] <= frame.shape[0] and emoji.x + emoji_resized.shape[1] <= frame.shape[1]:
                        frame[emoji.y:emoji.y + emoji_resized.shape[0], emoji.x:emoji.x + emoji_resized.shape[1], c] = \
                            emoji_resized[:, :, c] * (emoji_resized[:, :, 3] / 255.0) + \
                            frame[emoji.y:emoji.y + emoji_resized.shape[0], emoji.x:emoji.x + emoji_resized.shape[1], c] * (1.0 - emoji_resized[:, :, 3] / 255.0)

    return frame

### Filtro de cambio de pelo

In [4]:
import cv2
import mediapipe as mp
from mediapipe.tasks.python import vision
from mediapipe.tasks.python import BaseOptions
import numpy as np
import time

# Configuración del ImageSegmenter (para segmentación de cabello)
options = vision.ImageSegmenterOptions(
    base_options=BaseOptions(model_asset_path="models/hair_segmenter.tflite"),
    output_category_mask=True,
    running_mode=vision.RunningMode.LIVE_STREAM,
    result_callback=lambda result, output_image, timestamp_ms: setattr(SEGMENTER, 'last_result', result)
)
SEGMENTER = vision.ImageSegmenter.create_from_options(options)

def apply_hair_segmentation_filter(frame):
    # Reducción de resolución para mejorar la velocidad de procesamiento
    frame_resized = cv2.resize(frame, (640, 480))
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    frame_rgb = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
    prev_time = 0
    fps_limit = 15  # Limitar a 15 FPS para procesamiento

    # Control de FPS
    current_time = time.time()
    if (current_time - prev_time) * 1000 >= (1000 / fps_limit):
        SEGMENTER.segment_async(frame_rgb, time.time_ns() // 1_000_000)
        prev_time = current_time

    # Procesar y mostrar resultados si existen
    if hasattr(SEGMENTER, 'last_result') and SEGMENTER.last_result:
        segmentation_result = SEGMENTER.last_result
        category_mask = segmentation_result.category_mask.numpy_view()

        # Crear una máscara de color solo en la región de cabello (color morado)
        hair_color = (255, 0, 255)  # Morado en BGR
        color_mask = np.zeros_like(frame_resized)
        color_mask[category_mask == 1] = hair_color

        # Aplicar desenfoque a la máscara para suavizar los bordes
        blurred_mask = cv2.GaussianBlur(color_mask, (15, 15), 0)

        # Mezclar la imagen original con la máscara de color para resaltar el cabello
        final_frame = cv2.addWeighted(frame_resized, 1, blurred_mask, 0.4, 0)

        return final_frame
    else:
        return frame_resized  # Si no hay resultados, devolver el frame original



c:\Users\gatete\anaconda3\envs\VC_P5\Lib\site-packages\mediapipe\tasks\python\vision\image_segmenter.py:158: UserWarning: MessageFactory class is deprecated. Please use GetMessageClass() instead of MessageFactory.GetPrototype. MessageFactory class will be removed after 2024.
  graph_config = self._runner.get_graph_config()


In [5]:
def aplicar_filtro(frame, mode, points):
    with mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            min_detection_confidence=0.5) as face_mesh:
        if mode == 1:
            frame = process_frame_mode1(frame, face_mesh)
        elif mode == 2:        
            process_frame(frame, face_mesh, points, mode)
        return

def change_filter(frame, mode):
    if mode == 1: aplicar_filtro(frame, mode, [0, 17, 468, 473])
    elif mode == 2: aplicar_filtro(frame, mode, [127, 356, 183, 332])
    return

# Función principal
def main(): 
    cap = cv2.VideoCapture(0) 
    mode = 1  # Modo inicial
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
            
    if not cap.isOpened():
        print("No se puede abrir la cámara")
        exit()

    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        # Cambiar filtro según el modo
        if mode != 3: change_filter(frame, mode)
        elif mode == 3: frame = apply_hair_segmentation_filter(frame)
        
        # Mostramos el frame
        cv2.imshow('Frame', frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('1'): mode = 1
        elif key == ord('2'): mode = 2
        elif key == ord('3'): mode = 3
        elif key == ord('q'):  break
        
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()